<a href="https://colab.research.google.com/github/xwang222/My-Projects/blob/main/Python_Business_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Data Analytics

This notebook is a summary of codes commonly used for data cleaning and descriptive analysis, which is summartized across several fantastic notebooks here:

https://github.com/firmai/python-business-analytics

For detailed implementation and better demostration, please visit the github repository above. 

All credits should be given to the original author.

## Data Cleaning

### Handling Null Values

#### Overall Distribution of Null Values

In [ ]:
# Check to see if there are any missing values in our data set
df.isnull().any()

In [ ]:
# gives some infomation on columns types and number of null values
tab_info=pd.DataFrame(df_initial.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(df_initial.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info=tab_info.append(pd.DataFrame(df_initial.isnull().sum()/df_initial.shape[0]*100).T.
                         rename(index={0:'null values (%)'}))
print ('-' * 10 + " Display information about column types and number of null values " + '-' * 10 )
print 
display(tab_info)

#### Investigate Why there is missing value

After viewing the distribution of the null values, before deleting them, it might be a good idea to ask the following questions:

1. Why are they missing? Are they missing for valid reasons or are they missing because of some unexpected bugs in the code?

2. How are they missing? Are they missing at random or is missing correlated with some features?

Possible ways to go:

1. Drop them if the reason for missing is valid and there is nothing that you can do.
2. Impute them using some statistical methods (personally not recommend)
3. Encode a binary variable that shows the missing status

In [ ]:
# drop NA values that appeared in a particular column
df_initial.dropna(axis = 0, subset = ['CustomerID'], inplace = True)

### Handling Duplicates

In [ ]:
print('Duplicate data entries: {}'.format(df_initial.duplicated().sum()))
df_initial.drop_duplicates(inplace = True)

### Handling Outliers

In [ ]:
# box plot
sns.set(font_scale=1.0)
fig, axes = plt.subplots(nrows=3,ncols=2)
fig.set_size_inches(15, 15)
sns.boxplot(data=train,y="cnt",orient="v",ax=axes[0][0])
sns.boxplot(data=train,y="cnt",x="mnth",orient="v",ax=axes[0][1])
sns.boxplot(data=train,y="cnt",x="weathersit",orient="v",ax=axes[1][0])
sns.boxplot(data=train,y="cnt",x="workingday",orient="v",ax=axes[1][1])
sns.boxplot(data=train,y="cnt",x="hr",orient="v",ax=axes[2][0])
sns.boxplot(data=train,y="cnt",x="temp",orient="v",ax=axes[2][1])

axes[0][0].set(ylabel='Count',title="Box Plot On Count")
axes[0][1].set(xlabel='Month', ylabel='Count',title="Box Plot On Count Across Months")
axes[1][0].set(xlabel='Weather Situation', ylabel='Count',title="Box Plot On Count Across Weather Situations")
axes[1][1].set(xlabel='Working Day', ylabel='Count',title="Box Plot On Count Across Working Day")
axes[2][0].set(xlabel='Hour Of The Day', ylabel='Count',title="Box Plot On Count Across Hour Of The Day")
axes[2][1].set(xlabel='Temperature', ylabel='Count',title="Box Plot On Count Across Temperature")

In [ ]:
# standardization or IQR
q1 = train.cnt.quantile(0.25)
q3 = train.cnt.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 -(1.5 * iqr) 
upper_bound = q3 +(1.5 * iqr) 
train_preprocessed = train.loc[(train.cnt >= lower_bound) & (train.cnt <= upper_bound)]
print("Samples in train set without outliers: {}".format(len(train_preprocessed)))

### Restructing Data

In [ ]:
# Move the reponse variable "turnover" to the front of the table
front = df['turnover']
df.drop(labels=['turnover'], axis=1,inplace = True)
df.insert(0, 'turnover', front)
df.head()

In [ ]:
# customer visit, time series construct

# weekday
def add_day_of_week(index): 
    return (index) % 7

df['day_of_week'] = df['visit_day'].apply(add_day_of_week) # add new column for day of week
df.loc[df['day_of_week'] == 0, 'day_of_week'] = 7 # set all the 0's to 7 (Sunday)
df['day_of_week'] = df['day_of_week'].astype(np.uint8) # convert to uint8 to save memory

In [ ]:
# reshape wide to long
df_raw1 = (df_raw.merge(visits_df, right_index = True, left_index = True) # merge the original data with the expanded visits
                .drop(['visits'], axis=1) # drop the origil visits columns
                .melt(id_vars = ['visitor_id'], value_name = "visit_day") # transform from wide format to long format
                .drop("variable", axis = 1) # drop variable column which was added by melt function
                .dropna() # drop the missing entries
            )

In [ ]:
# shift data period
features_tot_dow = list(df_w2.columns[df_w2.columns.str.contains('tot_dow')].values)
features_prop_dow = list(df_w2.columns[df_w2.columns.str.contains('prop_dow')].values)
features_other = ['freq', 'weeks_since_prev_visit']
features = []
features.extend(features_tot_dow)
features.extend(features_prop_dow)
features.extend(features_other)
features

df_w2[features] = df_w2.groupby('visitor_id')[features].shift(1)

## Data Exploration

Remember to explore the constucted features after feature engineering step

### Explore Data Attributes

#### Some Commonly Used Commands

In [ ]:
# view 20 rows randomly sampled from the dataframe
df.sample(20)

# view 20 columns randomly sampled from the dataframe
df.nlargest(10,"age")

# sorting
df.sort_value("age",ascending=False).head(10)

df.info()

# obtain dimension
df.shape()

df.describe().T

# find the counts of NA
df.isna().sum()

# find unique value
df.nunique()

# string filter
df.name.str.contains("abc")

# select columns of a dataframe
df[["age","income"]]

df.query('city=="SH"')

df.sort_values(by=["city","age"])

np.where(condition,"Y","N")

# string operation
df.location.str.strip()

df.location.str.lstrip()

df.location.str.rstrip()

df.city.replace('A','B')

df.location.str.lower()

# dataframe merge/join
pd.merge(df1,df2,how="left",left_on="",right_on="")

#### (Number of) Unique Values of a column

In [ ]:
# show unique values
df_initial['Country'].value_counts()

# show how many unique values
len(df_initial['Country'].value_counts())
print('No. of cuntries in dataframe: {}'.format(len(df_initial['Country'].value_counts())))

#### Aggregation by Some Attribute

In [ ]:
# aggregate by country id, count the number of orders/customers
temp_no_of_order_per_count = df_initial[['CustomerID','Country']].groupby(['Country']).count()
temp_no_of_order_per_count = temp_no_of_order_per_count.reset_index(drop = False)

In [ ]:
# the number of products purchased in every transaction
temp = df_initial.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)['InvoiceDate'].count()

In [ ]:
# order cancel rate
nb_products_per_basket['order_cancelled'] = nb_products_per_basket['InvoiceNo'].apply(
    lambda x:int('C' in x))

n1 = nb_products_per_basket['order_cancelled'].sum()
n2 = nb_products_per_basket.shape[0]
percentage = (n1/n2)*100
print('Number of orders cancelled: {}/{} ({:.2f}%) '.format(n1, n2, percentage))

In [ ]:
# of visits and stats on cart amount / users
transactions_per_user=basket_price.groupby(by=['CustomerID'])['Basket Price'].agg(['count','min','max','mean','sum'])

In [ ]:
# accumulated sum
df_w1['freq'] = df_w1.groupby('visitor_id')['total_visits_in_week'].cumsum().astype(np.uint32)

In [ ]:
# pivot, customer segmentation check

### Correlation Analysis

In [ ]:
matrix = train[number_features + target].corr()
heat = np.array(matrix)
heat[np.tril_indices_from(heat)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sns.set(font_scale=1.0)
sns.heatmap(matrix, mask=heat,vmax=1.0, vmin=0.0, square=True,annot=True, cmap="Reds")

## Feature Engineering

#### Feature added using existing columns

In [ ]:
# the number of days elapsed since the first purchase (** FirstPurchase ) 
# and the number of days since the last purchase ( LastPurchase **)
last_date = basket_price['InvoiceDate'].max().date()

first_registration = pd.DataFrame(basket_price.groupby(by=['CustomerID'])['InvoiceDate'].min())
last_purchase      = pd.DataFrame(basket_price.groupby(by=['CustomerID'])['InvoiceDate'].max())

test  = first_registration.applymap(lambda x:(last_date - x.date()).days)
test2 = last_purchase.applymap(lambda x:(last_date - x.date()).days)

transactions_per_user.loc[:, 'LastPurchase'] = test2.reset_index(drop = False)['InvoiceDate']
transactions_per_user.loc[:, 'FirstPurchase'] = test.reset_index(drop = False)['InvoiceDate']


#### Feature added using other machine learning tools

In [ ]:
# customer segmentation, creation of customer clusters